In [31]:
! pip install selenium
! pip install html_table_parser

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for beautifulsoup4==4.4.1 from https://files.pythonhosted.org/packages/cf/69/9abfdab06490af5e0233bcebe3f617ec128486d94ea987ad4f77b9332eef/beautifulsoup4-4.4.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/81.4 kB ? eta -:--:--
   --------------- ------------------------ 30.7/81.4 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 81.4/81.4 kB 1.5 MB/s eta 0:00:00
  Created wheel for html_table_parser: filename=html_table_parser-0.1.0-py3-none-any.whl size=4638 sha256=2f599eac9f28c620288fd47ba00512a4bafaa8e8464d32c88e500e5168094cc8
  Stored in directory: c:\users\melid\appdata\local\pip\cache\wheels\3f\61\3e\205c3503f7c244357c1ba105368ba70095a71a075624689850
Successfully built html_table_parser
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.12.2
    Uninstalling beau

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [2]:
driver = webdriver.Chrome()

In [3]:
url = "https://statiz.sporki.com/stats/?m=main&m2=all"
driver.get(url)

In [4]:
driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/section/div[1]/div/button[4]').click()

# 팀 옵션 드롭다운 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/button').click()
# 삼성 라이온즈 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/ul/li[5]').click()

# 수비 > 포수 선택
driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/section/div[6]/ul/li[2]/a').click()

In [5]:
from urllib.request import urlopen

import collections
collections.Callable = collections.abc.Callable

from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import pandas as pd

In [6]:
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

In [7]:
temp = soup.find_all('table')
p = parser.make2d(temp[0])    
for i in range(len(p)):
    print(i, p[i])
print(len(p))

0 ['Rank', 'Name', 'Team', 'Sort▼', 'G', 'GS', 'IP', 'WP', 'PB', 'Pass9', '주자 견제', '주자 견제', '주자 견제', '주자 견제', '주자 견제', '도루 허용 상황', '도루 허용 상황', '도루 허용 상황', '도루 허용 상황', '도루 저지 상황', '도루 저지 상황', '도루 저지 상황', '도루 저지 상황', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도', '수비 관련 득점 기여도']
1 ['Rank', 'Name', 'Team', '수비 관련 득점 기여도-종합 RAA', 'G', 'GS', 'IP', 'WP', 'PB', 'Pass9', 'SB', 'CS', 'CS%', 'SBA', 'SBA%', '2루', '3루', '홈', '더블', '2루', '3루', '홈', '더블', 'Range RAA', 'Err RAA', 'CS RAA', 'Blk RAA', 'Frm RAA', '종합 RAA▼', '/144']
2 ['1', '김기연', '24  C', '0.66', '17', '10', '108.0', '6', '0', '0.500', '4', '2', '33.3', '168', '3.6', '2', '2', '0', '0', '2', '0', '0', '0', '0.25', '-0.51', '0.19', '0.72', '', '0.66', '5.28']
3 ['2', '안승한', '24  C', '0.58', '2', '1', '9.1', '0', '0', '0.000', '1', '0', '0.0', '17', '5.9', '1', '0', '0', '0', '0', '0', '0', '0', '0.00', '0.04', '0.00', '0.54', '', '0.58', '20.79']
4 ['3', '장승현', '24  C', '0.55', '9', '7', 

In [8]:
num = (len(p) // 10)

start_num = 2
for i in range(num*2+1, 1, -1):
    n = start_num * 10 + (i-2)
    print(n)
    if i % 2 == 0:
        start_num -= 1

In [11]:
data=pd.DataFrame(p)
data=data[2:]
data.columns=["Rank", "Name", "Position", "Total RAA2","G", "GS", "IP", "WP", "PB", "Pass9", "SB", "CS", "CS%", "SBA", "SBA%", "SB 2", "SB 3", "SB H", "SB Double", "CS 2", "CS 3", "CS H", "CS Double", "Range RAA", "Err RAA", "CS RAA", "Blk RAA", "Frm RAA","Total RAA", "/144"]
data.drop(columns=['Total RAA2'], inplace=True)
data['Team'] = "Doosan"
data['Position'].replace({"24  ":""}, regex=True, inplace=True)
print(data)

  Rank Name Position   G  GS     IP  WP PB  Pass9  SB  ... CS H CS Double  \
2    1  김기연        C  17  10  108.0   6  0  0.500   4  ...    0         0   
3    2  안승한        C   2   1    9.1   0  0  0.000   1  ...    0         0   
4    3  장승현        C   9   7   58.0   4  0  0.621   3  ...    0         0   
5    4  윤준호        C   1   0    2.0   1  0  4.500   0  ...    0         0   
6    5  양의지        C  25  25  202.2  12  2  0.622  16  ...    0         0   

  Range RAA Err RAA CS RAA Blk RAA Frm RAA Total RAA    /144    Team  
2      0.25   -0.51   0.19    0.72              0.66    5.28  Doosan  
3      0.00    0.04   0.00    0.54              0.58   20.79  Doosan  
4     -0.15    0.26   0.00    0.37              0.55    8.73  Doosan  
5      0.00    0.01          -0.51             -0.50  -71.61  Doosan  
6     -0.12    0.42  -0.19   -1.12             -0.98   -3.72  Doosan  

[5 rows x 30 columns]


In [12]:
data.to_csv("Doosan_field_C.csv", index=False)